# Load_Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import struct
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report,ConfusionMatrixDisplay,confusion_matrix
import tensorflow as tf
import torch
from torch import nn , optim
from torchvision import datasets, transforms

In [ ]:
class DataLoading(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            _,_ = struct.unpack('>2I',file.read(8))
            labels = bytearray(file.read())        
        with open(images_filepath, 'rb') as file:
            _,size,rows,cols = struct.unpack(">4I", file.read(16))
            image_data = bytearray(file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return images, labels


    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)
        

    def prepare_data(self):
      (x_train, y_train), (x_test, y_test) = self.load_data()
      x_train = np.array(x_train).reshape(60000,784)
      x_test = np.array(x_test).reshape(10000,784)
      X_train = pd.DataFrame(x_train)
      X_test = pd.DataFrame(x_test)
      y_train = pd.DataFrame(np.array(y_train).flatten())
      y_test = pd.DataFrame(np.array(y_test).flatten())
      y_train.rename(columns={0:'target'},inplace =True)
      y_test.rename(columns={0:'target'},inplace =True)
      return (X_train,y_train),(X_test,y_test)
    
    def info(self):
     (X_train, y_train), (X_test, y_test) = self.prepare_data()
     print('Train_X_shape: {0} , Test_X_shape: {1}'.format(X_train.shape,X_test.shape))
     print('-------------------------------------------------')
     print('Train_y_shape: {0} , Test_y_shape: {1}'.format(y_train.shape,y_test.shape))
     print('-------------------------------------------------')
     print('5_rows_training: ')
     print(X_train.head())
     print('-------------------------------------------------')
     print(y_train.head())
     print('-------------------------------------------------')
     print('5_rows_testing: ')
     print(X_test.head())
     print('-------------------------------------------------')
     print(y_test.head())
     print('-------------------------------------------------')
     print('labels_numbers: ')
     print(y_train.value_counts())
     print('-------------------------------------------------')
     print(y_test.value_counts())
     
    def show_image(self,number_row):
      (X_train, y_train), (X_test, y_test) = self.prepare_data()
      combine_X = pd.concat([X_train,X_test])
      combine_y = pd.concat([y_train,y_test])
      print('-------------------------------------------------')
      print('label: {0}'.format(combine_y.iloc[number_row][0]))
      print()
      plt.imshow(combine_X.iloc[number_row,::].to_numpy().reshape(28,28),cmap = plt.cm.gray)
    

In [ ]:
training_images_filepath = '/kaggle/input/mnist-dataset/train-images.idx3-ubyte'
training_labels_filepath = '/kaggle/input/mnist-dataset/train-labels.idx1-ubyte'
test_images_filepath = '/kaggle/input/mnist-dataset/t10k-images.idx3-ubyte'
test_labels_filepath = '/kaggle/input/mnist-dataset/t10k-labels.idx1-ubyte'

# load MINST dataset

mnist_dataloader = DataLoading(training_images_filepath, training_labels_filepath,
                               test_images_filepath, test_labels_filepath)
(X_train, y_train), (X_test, y_test) = mnist_dataloader.prepare_data()
# info
mnist_dataloader.info()

# show image
mnist_dataloader.show_image(10000)


# Sickit-Learn

In [ ]:
class SickitAlgorithms(object):
  def knn_model(slef,k_neighbors:int):
    knn = KNeighborsClassifier(n_neighbors=k_neighbors)
    knn.fit(X_train,y_train)
    print()
    print('----------------          KNN      -------------------')
    print()
    print('test_score: {0}'.format(np.round(knn.score(X_test,y_test)*100,2)))
    print('---------------------------------------------------------')
    print('classification_report: ')
    print(classification_report(y_test,knn.predict(X_test)))
    print('---------------------------------------------------------')
    print('Confusion_matrix: ')
    ConfusionMatrixDisplay(confusion_matrix(y_test,knn.predict(X_test))).plot()
    plt.show()
    print('---------------------------------------------------------')



  def svc_model(slef,c:int):
    svc = SVC(C=c)
    svc.fit(X_train,y_train)
    print()
    print('----------------          SVC      -------------------')
    print()
    print('test_score: {0}'.format(np.round(svc.score(X_test,y_test)*100,2)))
    print('---------------------------------------------------------')
    print('classification_report: ')
    print(classification_report(y_test,svc.predict(X_test)))
    print('---------------------------------------------------------')
    print('Confusion_matrix: ')
    ConfusionMatrixDisplay(confusion_matrix(y_test,svc.predict(X_test))).plot()
    plt.show()
    print('---------------------------------------------------------')



  def mlp_model(self,hidden_layer_size:tuple,active:str,optimizer:str):
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_size,activation = active,solver=optimizer)
    mlp.fit(X_train,y_train)
    print()
    print('----------------          MLP      -------------------')
    print()
    print('test_score: {0}'.format(np.round(mlp.score(X_test,y_test)*100,2)))
    print('---------------------------------------------------------')
    print('classification_report: ')
    print(classification_report(y_test,mlp.predict(X_test)))
    print('---------------------------------------------------------')
    print('Confusion_matrix: ')
    ConfusionMatrixDisplay(confusion_matrix(y_test,mlp.predict(X_test))).plot()
    plt.show()
    print('---------------------------------------------------------')


             
 

In [ ]:
algorithm_training = SickitAlgorithms()
algorithm_training.knn_model(5)
algorithm_training.svc_model(100)
algorithm_training.mlp_model((10,5),'tanh','sgd')

# TensorFlow

In [ ]:
# Define the model architecture
model = tf.keras.Sequential([
  tf.keras.layers.Flatten(), # input
  tf.keras.layers.Dense(128, activation='relu'),# 1 hidden layer with 128 neurons
  tf.keras.layers.Dense(10) # output after 10 epoch
])

# Compile the model
model.compile(optimizer='adam', # using adam 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # sparse categorical loss
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('----------------------------------')
print('Test accuracy:', test_acc)
print('Test loss:',test_loss)

# PyTorch

In [ ]:
# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512) # size , number of neurons
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = x.view(-1, 784) # reshaping input 
        x = nn.functional.tanh(self.fc1(x)) # pass input to relu/tanh activation function
        x = nn.functional.tanh(self.fc2(x)) # pass hidden to relu/tanh activation function
        x = self.fc3(x) 
        return nn.functional.log_softmax(x, dim=1) # output that will be from [0----->9] after doing log_softmax 
           
# Define the training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data) # forward propagation
        loss = nn.functional.cross_entropy(output, target)
        loss.backward() # backward propagation
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# Define the test function
def test(model, device, test_loader):
    model.eval() # evaluation mode
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.functional.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            # compares the predictions to the true labels,
            # and adds the number of correct predictions to the correct variable to get percentage .

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100 * correct / len(test_loader.dataset)))



In [ ]:
# Set up the device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the MNIST dataset and apply transformations
train_dataset = datasets.MNIST('data', train=True, download=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.1307,), (0.3081,))
                             ]))
test_dataset = datasets.MNIST('data', train=False, download=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                            ]))

# Set up the data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64 ,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=True)

# Instantiate the neural network and move it to the device
model = Net().to(device)

# Set up the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Train and test the model for 10 epochs
for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)

test(model, device, test_loader)